# kv-cache

视频一：https://www.bilibili.com/video/BV17CPkeEEzk/?spm_id_from=333.337.search-card.all.click&vd_source=071b23b9c7175dbaf674c65294124341  
视频二：https://www.bilibili.com/video/BV1dN4y1y7e4/?spm_id_from=333.337.search-card.all.click&vd_source=071b23b9c7175dbaf674c65294124341  

博客一：https://mingchao.wang/Ao5PjBRP/

llama官网源码：https://github.com/meta-llama/llama/blob/main/llama/model.py

本地代码：参考./大模型从零构建/Andrej_Karpathy/GPT-2_124M/5-train_gpt2_应用kv-cache.py中的实现，主要修改为注意力部分（加上调用了注意力部分的模块）以及generate函数。


## 优势：
1、避免重复计算  
不使用kv-cache的计算过程如下，每次生成一个新 token 时，都需要重新计算所有之前 token 的 Key 和 Value：

<div align="center">
    <img src="kv-cache/不使用kv-cache.png" alt="图片描述" width="800"/>
</div>


## 问题：
1、为什么kv-cache只存在于decoder中？

<div align="center">
    <img src="kv-cache/只存在于decoder.png" alt="图片描述" width="800"/>
</div>

（1）自回归特性：Decoder 的核心特点是自回归（Auto-Regressive），即每次生成一个新的 token 时，只能依赖于之前已经生成的 token。这种机制使得 Decoder 在推理过程中是逐 token 生成的。而Encoder中，每个token的计算都是独立的，并行计算，不需要依赖前面和后续的token。
（2）Masked Attention：Decoder 使用 Masked Attention，确保当前 token 的生成不会受到未来 token 的影响。在Encoder中，计算每个token时都需要与整个句子的信息进行交互。


# vLLM
用vLLM解决kv-cache存在的问题

视频：https://www.bilibili.com/video/BV1kx4y1x7bu/?spm_id_from=333.337.search-card.all.click&vd_source=071b23b9c7175dbaf674c65294124341

kv-cache存在的三种浪费，利用率20%-40%：  
1、预分配，但不会用到；  
2、预分配，但尚未用到；  
3、显存之间的间隔碎片，不足以分配给下一个文本生成。

vLLM解决方法：  
1、Page Attention，内存分块，利用逻辑内存和物理显存，利用率达到96%；  
2、Sharing KV Blocks：用一个prompt生成多个output时的优化。